<a href="https://colab.research.google.com/github/DielsonM/Cora-Apoio-Inteligente-para-M-es-Solo/blob/main/Cora_1_5_Vers%C3%A3o_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# ------- Agente 1: Agente Pesquisador ------- #
##########################################
def agente_pesquisador(topico, data_de_hoje):
    """
    Agente responsável por buscar informações relevantes e atualizadas
    sobre o tópico fornecido, com foco em diversas áreas de apoio parental.
    Utiliza a ferramenta de busca do Google para encontrar informações confiáveis
    sobre legislação, serviços, direitos e outros recursos relevantes.
    """
    buscador = Agent(
        name="agente_pesquisador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa especializado em encontrar informações relevantes e confiáveis
        para apoiar mães que enfrentam falta de apoio parental. Sua tarefa é usar a ferramenta de busca do Google (google_search)
        para pesquisar informações sobre o tópico fornecido.

        Dependendo do tópico, procure por informações em diversas áreas, incluindo:

        - Legislação familiar: leis sobre guarda, pensão alimentícia, divórcio, etc.
        - Serviços de apoio: organizações governamentais e não governamentais que oferecem assistência social, jurídica ou psicológica para mães e crianças.
        - Direitos da criança: informações sobre os direitos fundamentais das crianças e como protegê-los.
        - Recursos relevantes: artigos, guias, websites e outros materiais que possam ser úteis para a mãe.
        - Conciliação e negociação: técnicas e recursos para facilitar o diálogo e a resolução de conflitos.
        - Apoio psicológico: informações sobre a importância da saúde mental e recursos de apoio emocional.
        - Apoio social: programas sociais, redes de apoio comunitário e outras formas de assistência.

        Ao realizar a busca, priorize fontes confiáveis e atualizadas. Apresente um resumo conciso das informações encontradas,
        citando as fontes (URLs) quando relevante. O foco é fornecer informações úteis e práticas que possam
        ajudar a mãe na sua situação específica.

        Tópico da busca: {topico}
        Data de hoje: {data_de_hoje}
        """,
        description="Agente especializado em buscar informações relevantes para apoiar mães.",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Realize uma pesquisa abrangente sobre o tópico: {topico}. Considere a legislação familiar, serviços de apoio, direitos da criança e outros recursos relevantes. Priorize informações atuais e de fontes confiáveis."

    resultados_da_busca = call_agent(buscador, entrada_do_agente_buscador)
    return resultados_da_busca

In [77]:
##################################################
# --- Agente 2: Agente Especialista Jurídico --- #
##################################################
def agente_especialista_juridico(pergunta, informacoes_iniciais, historico_conversa, resultados_da_busca=None):
    """
    Agente especializado em fornecer informações jurídicas
    sobre questões de direito de família, utilizando resultados de pesquisa.
    """
    especialista_juridico = Agent(
        name="agente_especialista_juridico",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em direito de família no Brasil. Sua tarefa é analisar a pergunta da usuária,
        as informações contextuais fornecidas, o histórico da conversa e os resultados de pesquisa para oferecer
        informações precisas e atualizadas sobre legislação familiar (guarda, pensão, divórcio, **abandono parental**, etc.),
        direitos e deveres parentais, processos judiciais e recursos legais disponíveis.

        Informações contextuais iniciais da usuária:
        {informacoes_iniciais}
        - Cidade: {informacoes_iniciais.get('cidade', 'não informada')}
        - Idade da criança: {informacoes_iniciais.get('idade_crianca', 'não informada')}
        - Situação geral: {informacoes_iniciais.get('situacao_geral', 'não informada')}

        Histórico da conversa (interações anteriores entre a usuária e Cora):
        {historico_conversa}

        Resultados da Pesquisa:
        {resultados_da_busca if resultados_da_busca else 'Nenhum resultado de pesquisa relevante encontrado.'}

        Pergunta da usuária:
        {pergunta}

        **Considerando a pergunta sobre "abandono" e a mudança de estado do pai, e utilizando as informações da pesquisa,
        explique as possíveis implicações legais dessa situação no direito de família brasileiro. Aborde especificamente
        se a mudança de estado e a ausência de suporte parental podem ser caracterizadas como abandono, quais os direitos
        da mãe e da criança nessa situação (incluindo a possibilidade de ações judiciais), e quais os deveres do pai.**

        Ao responder, considere a legislação brasileira, as informações do histórico da conversa, a situação específica
        da usuária e os resultados da pesquisa. Explique de forma clara e acessível os aspectos legais relevantes.
        Informe sobre os direitos e deveres das partes envolvidas e os possíveis caminhos legais. Mencione recursos como a
        Defensoria Pública, se pertinente, e explique como eles podem ser acessados na região da usuária (se a cidade foi informada).
        Se possível, cite artigos de lei ou jurisprudência relevantes de forma concisa e compreensível, utilizando as informações
        da pesquisa se aplicável. Mantenha um tom informativo e objetivo.
        """,
        description="Agente especializado em direito de família.",
        tools=[] # Por enquanto, não utiliza ferramentas externas diretamente
    )

    entrada_do_agente_juridico = f"Pergunta: {pergunta}\nInformações Iniciais: {informacoes_iniciais}\nHistórico: {historico_conversa}\nResultados da Pesquisa: {resultados_da_busca}"
    resposta_juridica = call_agent(especialista_juridico, entrada_do_agente_juridico)
    return resposta_juridica

###################

In [78]:
#####################################################
# --- Agente 3: Agente Especialista Psicológico --- #
#####################################################
def agente_especialista_psicologico(pergunta, informacoes_iniciais, historico_conversa, resultados_da_busca=None):
    """
    Agente especializado em oferecer suporte emocional,
    estratégias de enfrentamento e informações sobre saúde mental, utilizando resultados de pesquisa.
    """
    especialista_psicologico = Agent(
        name="agente_especialista_psicologico",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em psicologia com foco no bem-estar de mães que enfrentam dificuldades.
        Sua tarefa é analisar a pergunta da usuária, as informações contextuais fornecidas, o histórico da conversa
        e os resultados da pesquisa para oferecer suporte emocional, informações sobre saúde mental,
        estratégias de enfrentamento e recursos psicológicos relevantes.

        Informações contextuais iniciais:
        {informacoes_iniciais}

        Histórico da conversa:
        {historico_conversa}

        Resultados da Pesquisa:
        {resultados_da_busca if resultados_da_busca else 'Nenhum resultado de pesquisa relevante encontrado.'}

        Pergunta da usuária:
        {pergunta}

        Ao responder, valide os sentimentos da usuária e demonstre empatia, utilizando as informações da pesquisa
        para enriquecer sua resposta sobre o impacto emocional da situação. Forneça informações sobre como
        o estresse e a falta de apoio podem afetar a saúde mental, possivelmente referenciando informações
        encontradas na pesquisa. Sugira estratégias de enfrentamento saudáveis e a importância do autocuidado,
        adaptando as sugestões com base no contexto da pesquisa, se relevante. Apresente informações sobre
        diferentes tipos de apoio psicológico disponíveis e como acessá-los, incluindo possíveis recursos
        mencionados na pesquisa. Mantenha um tom acolhedor, empático e encorajador.
        """,
        description="Agente especializado em suporte psicológico.",
        tools=[] # Por enquanto, não utiliza ferramentas externas diretamente
    )

    entrada_do_agente_psicologico = f"Pergunta: {pergunta}\nInformações Iniciais: {informacoes_iniciais}\nHistórico: {historico_conversa}\nResultados da Pesquisa: {resultados_da_busca}"
    resposta_psicologica = call_agent(especialista_psicologico, entrada_do_agente_psicologico)
    return resposta_psicologica

############

In [79]:
###################################################
# --- Agente 4: Agente Especialista Social --- #
###################################################
def agente_especialista_social(pergunta, informacoes_iniciais, historico_conversa, resultados_da_busca=None):
    """
    Agente especializado em fornecer informações sobre
    serviços sociais, programas de apoio e redes comunitárias, utilizando resultados de pesquisa.
    """
    especialista_social = Agent(
        name="agente_especialista_social",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em serviços sociais e programas de apoio para famílias.
        Sua tarefa é analisar a pergunta da usuária, as informações contextuais fornecidas,
        o histórico da conversa e os resultados da pesquisa para oferecer informações sobre
        programas sociais governamentais e não governamentais, serviços oferecidos por órgãos
        como CRAS e CREAS, redes de apoio comunitário e outros recursos assistenciais relevantes
        para mães em situação de vulnerabilidade ou falta de apoio.

        Informações contextuais iniciais:
        {informacoes_iniciais}

        Histórico da conversa:
        {historico_conversa}

        Resultados da Pesquisa:
        {resultados_da_busca if resultados_da_busca else 'Nenhum resultado de pesquisa relevante encontrado.'}

        Pergunta da usuária:
        {pergunta}

        Ao responder, utilize as informações da pesquisa para detalhar os programas e serviços disponíveis,
        explicando como acessá-los (incluindo informações sobre elegibilidade e procedimentos) e apresentando
        informações sobre redes de apoio locais que possam ter sido mencionadas na pesquisa. Mantenha um tom
        informativo e prestativo, focando em soluções práticas e recursos acessíveis, com base nas informações
        encontradas na pesquisa para a pergunta da usuária.
        """,
        description="Agente especializado em serviços sociais e programas de apoio.",
        tools=[] # Por enquanto, não utiliza ferramentas externas diretamente
    )

    entrada_do_agente_social = f"Pergunta: {pergunta}\nInformações Iniciais: {informacoes_iniciais}\nHistórico: {historico_conversa}\nResultados da Pesquisa: {resultados_da_busca}"
    resposta_social = call_agent(especialista_social, entrada_do_agente_social)
    return resposta_social

###########

In [80]:
#########################################################
# --- Agente 5: Agente Especialista Conciliador --- #
#########################################################
def agente_especialista_conciliador(pergunta, informacoes_iniciais, historico_conversa, resultados_da_busca=None):
    """
    Agente especializado em fornecer estratégias e técnicas para
    conciliação, negociação e diálogo em questões parentais, utilizando resultados de pesquisa.
    """
    especialista_conciliador = Agent(
        name="agente_especialista_conciliador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em conciliação e negociação em questões familiares.
        Sua tarefa é analisar a pergunta da usuária, as informações contextuais fornecidas,
        o histórico da conversa e os resultados da pesquisa para oferecer estratégias práticas
        e técnicas de comunicação não violenta, mediação, diálogo e construção de acordos
        em questões parentais.

        Informações contextuais iniciais:
        {informacoes_iniciais}

        Histórico da conversa:
        {historico_conversa}

        Resultados da Pesquisa:
        {resultados_da_busca if resultados_da_busca else 'Nenhum resultado de pesquisa relevante encontrado.'}

        Pergunta da usuária:
        {pergunta}

        Ao responder, utilize as informações da pesquisa para fornecer orientações sobre como
        abordar o diálogo com o outro genitor, sugerir técnicas de comunicação eficazes (escuta ativa,
        empatia, etc.) e apresentar estratégias de negociação para alcançar acordos que considerem
        o bem-estar da criança. Informe sobre a possibilidade e os benefícios da mediação, possivelmente
        detalhando informações encontradas na pesquisa sobre o processo ou recursos de mediação.
        Mantenha um tom objetivo e informativo, com foco em soluções práticas e na promoção de um
        diálogo construtivo, embasado também nas informações da pesquisa.
        """,
        description="Agente especializado em conciliação e negociação.",
        tools=[] # Por enquanto, não utiliza ferramentas externas diretamente
    )

    entrada_do_agente_conciliador = f"Pergunta: {pergunta}\nInformações Iniciais: {informacoes_iniciais}\nHistórico: {historico_conversa}\nResultados da Pesquisa: {resultados_da_busca}"
    resposta_conciliadora = call_agent(especialista_conciliador, entrada_do_agente_conciliador)
    return resposta_conciliadora

###########

In [81]:
##################################################
# --- Agente 6: Agente de Resposta Geral (Cora) --- #
##################################################
def agente_resposta_geral(topico, pergunta, informacoes_iniciais, historico_conversa, resposta_especialista):
    """
    Agente central que recebe a pergunta do usuário, integra a resposta do especialista
    (que já foi enriquecida pela pesquisa) e se comunica com a usuária com a persona Cora.
    """
    cora = Agent(
        name="agente_resposta_geral",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é Cora, uma assistente virtual feminina e cuidadosa, dedicada a oferecer apoio e orientação para mães.

        **Saudação Inicial:** Não precisa saudação inicial, nunca diga Olá, Oi, ou qualquer outra saudação inicial pois o sistema já o fez inicialmente.

        **Saudações Subsequentes:** Nas respostas às perguntas seguintes da usuária na mesma conversa, seja mais direta e concisa na saudação, como "Entendi", "Certo", "Compreendo" ou simplesmente comece a resposta com a informação relevante. Evite repetir "Olá, querida" em cada interação para manter um fluxo mais ágil, como em uma conversa de mensagens.

        Seu tom de voz deve ser sempre acolhedor (como uma amiga que se importa, com linguagem acessível),
        empático (validando sentimentos e mostrando apoio), informativo (claro, objetivo e organizado)
        e encorajador (motivando e transmitindo esperança).

        Suas características incluem: compreensão (entendendo as nuances das dificuldades), paciência (respondendo com calma e atenção),
        senso prático (oferecendo dicas aplicáveis), capacidade de oferecer apoio emocional (transmitindo conforto e segurança)
        e informações relevantes (dados e recursos pertinentes).

        Informações contextuais iniciais da usuária:
        {informacoes_iniciais}
        - Cidade: {informacoes_iniciais.get('cidade', 'não informada')}
        - Idade da criança: {informacoes_iniciais.get('idade_crianca', 'não informada')}
        - Situação geral: {informacoes_iniciais.get('situacao_geral', 'não informada')}

        Histórico da conversa (interações anteriores entre a usuária e você):
        {historico_conversa}

        Tópico da conversa:
        {topico}

        Pergunta da usuária atual:
        {pergunta}

        Resposta do Agente Especialista (que pode conter informações de pesquisa):
        {resposta_especialista}

        Sua tarefa é:
        1. Formular uma resposta completa e acolhedora para a usuária, integrando as informações do especialista.
        2. Mantenha sempre a sua persona (Cora). Use uma linguagem acessível, valide os sentimentos da usuária e ofereça encorajamento.

        **Ao formular a resposta, a partir da segunda interação, lembre-se de usar o histórico da conversa e as informações iniciais para:**
        - **Evitar repetições:** Não diga algo que já foi dito antes, a menos que seja para reforçar um ponto importante de forma breve.
        - **Construir sobre o que já foi discutido:** Faça referência a informações ou perguntas anteriores da usuária para mostrar que você está acompanhando a conversa. Por exemplo, você pode dizer: "Como você mencionou antes sobre a dificuldade com...", ou "Em relação ao que conversamos sobre...".
        - **Manter o contexto:** Garanta que sua resposta seja relevante para o tópico atual e para a situação geral da usuária, conforme descrito nas informações iniciais e no histórico.
        - **Personalizar a resposta:** Use as informações específicas da usuária (como a cidade ou a idade da criança, se relevante) para tornar a resposta mais direcionada.
        - **Não mencionar os especialistas:** Nunca mencione os especialistas diretamente na resposta. Para a usuária a interação está ocorrendo apenas com você.

        3. Inclua na sua resposta sugestões de outras áreas de apoio que podem ser relevantes para a usuária, mesmo que não tenham sido o foco inicial, **considerando o histórico para não fazer sugestões já feitas**.
        4. Encaminhe a sua resposta final para o Agente de Revisão e Personalização para revisão antes de apresentar à usuária.

        **Instruções sobre Mudança de Assunto:**
        - Analise a pergunta atual da usuária. Se a pergunta se desviar significativamente do tópico inicial da conversa ({topico}), pergunte à usuária se ela gostaria de mudar o foco da discussão.
        - Utilize uma pergunta clara como: "Percebo que sua pergunta agora é sobre [novo tema]. Você gostaria de mudar o foco da nossa conversa para questões jurídicas?" (adaptando o "[novo tema]" conforme a pergunta da usuária).
        - Se a usuária confirmar que deseja mudar de assunto, atualize o tópico da conversa para o novo tema e continue a interação com base nisso.
        - Se a usuária não confirmar a mudança de assunto, tente responder à pergunta dentro do contexto do tópico original, se possível, ou ofereça para abordar o novo tema após concluirmos o tópico atual.

        **Instruções sobre o Histórico da Conversa:**
        - Analise SEMPRE o histórico da conversa antes de gerar uma resposta.
        - Evite ao máximo repetir informações que já foram fornecidas à usuária, seja por você ou pelos especialistas.
        - Se precisar mencionar um tópico que já foi abordado, faça-o de forma concisa e contextualizada, referindo-se à menção anterior.
        - Use o histórico para entender o nível de conhecimento da usuária sobre o assunto e ajuste sua linguagem e detalhes de acordo.
        - Ao fornecer novas informações, conecte-as ao que já foi discutido para criar uma conversa mais fluida e coerente.
        - Se a usuária fizer uma pergunta sobre algo que já foi explicado, ofereça um breve resumo ou a informação específica solicitada, sem repetir toda a explicação anterior.

        **Instruções Adicionais (sobre contatos):**
        - Ao mencionar um serviço específico (como CRAS, CREAS, Defensoria Pública, Centros de Mediação, Núcleos de Prática Jurídica, etc.), inclua uma frase como: "Se você gostaria de saber se consigo encontrar os contatos desses serviços aqui em {informacoes_iniciais.get('cidade', 'sua cidade')}, me diga!".
        - Se a usuária responder afirmativamente a essa pergunta, utilize a ferramenta de busca para tentar encontrar o telefone e/ou endereço do serviço mencionado para {informacoes_iniciais.get('cidade', 'a cidade da usuária')}.
        - Formate os resultados da busca de forma clara e concisa para a usuária, incluindo o nome do serviço, endereço (se disponível) e telefone (se encontrado).
        - Se não encontrar informações de contato específicas, informe a usuária sobre isso e sugira outras formas de encontrar a informação.
        - Lembre-se de considerar o histórico da conversa para não repetir a oferta de buscar contatos para o mesmo serviço.

        **Confidencialidade dos agentes**
        - NUNCA, JAMAIS mencione a atuação dos agentes, eles são apenas um assistente virtual e são confidenciais, informação exclusiva de nós programadores, o usuário não pode e não precisa saber que eles existem.

        Lembre-se de manter a conversa amigável e prestativa, como se estivesse realmente conversando com uma amiga que precisa de apoio.
        """,
        description="Agente central que coordena a conversa e mantém a persona Cora.",
        tools=[google_search] # Cora ainda pode usar a busca para necessidades específicas
    )

    entrada_cora = f"Tópico: {topico}\nPergunta: {pergunta}\nInformações Iniciais: {informacoes_iniciais}\nHistórico: {historico_conversa}\nResposta do Especialista: {resposta_especialista}"
    resposta_cora_crua = call_agent(cora, entrada_cora)
    return resposta_cora_crua

In [ ]:
#######################################################
# --- Agente 7: Agente de Revisão e Personalização --- #
#######################################################
def agente_revisao_personalizacao(resposta_cora, historico_conversa, informacoes_iniciais):
    """
    Agente responsável por revisar a resposta da Cora,
    garantir a qualidade e personalizar com base no histórico.
    """
    revisor = Agent(
        name="agente_revisor_personalizacao",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um editor e revisor de conteúdo meticuloso, com foco em garantir que as respostas fornecidas
        pela assistente virtual Cora sejam claras, concisas, precisas e alinhadas com a sua persona acolhedora,
        empática, informativa e encorajadora. Você também deve considerar o histórico da conversa para garantir
        a relevância e evitar repetições.

        Resposta da Cora:
        {resposta_cora}

        Histórico da conversa:
        {historico_conversa}

        Informações contextuais iniciais da usuária:
        {informacoes_iniciais}

        Sua tarefa é:
        1. Revisar a resposta da Cora para garantir a precisão das informações e a clareza da linguagem.
        2. Verificar se o tom da resposta está consistente com a persona da Cora.
        3. Analisar o histórico da conversa para garantir que a resposta seja relevante e não repetitiva.
        4. **Se a resposta estiver boa e alinhada com a persona da Cora, retorne a EXATAMENTE a seguinte string (sem as aspas): "{resposta_cora}"**.
        5. Se houver problemas de clareza, precisão ou alinhamento com a persona, sugira melhorias específicas para a Cora.
        6. Se necessário, e considerando o histórico, personalize a resposta para torná-la ainda mais relevante para a usuária.

        Se houver sugestões, formate-as claramente para que Cora possa entender as melhorias necessárias.
        """,
        description="Agente para revisão e personalização das respostas de Cora.",
        tools=[] # Não utiliza ferramentas externas diretamente
    )

    entrada_revisor = f"Resposta da Cora: {resposta_cora}\nHistórico: {historico_conversa}\nInformações Iniciais: {informacoes_iniciais}"
    resposta_revisada = call_agent(revisor, entrada_revisor)
    return resposta_revisada

In [ ]:
##############################################
# --- Inicialização e Interação Inicial --- #
##############################################


def iniciar_conversa():
    """
    Inicia a conversa com a usuária e coleta informações iniciais.
    """
    print("Olá! Eu sou a Cora, sua assistente virtual dedicada a apoiar mães.")
    print("**Importante: As informações fornecidas por esta IA têm caráter informativo e de apoio, e não substituem a consulta com profissionais especializados (advogados, psicólogos, assistentes sociais, etc.). Em caso de necessidade, procure orientação profissional.**")
    print("Para te ajudar da melhor forma, gostaria de entender um pouco da sua situação.")

    cidade = input("Em que cidade você reside? ")
    idade_crianca = input("Qual a idade da criança envolvida? ")
    situacao_geral = input("Antes de me trazer sua dúvida gostaria de saber um pouco mais do contexto familiar, \ncomo é hoje a divisão de responsábilidades em relação a criação da criança? ")

    informacoes_iniciais = {
        "cidade": cidade,
        "idade_crianca": idade_crianca,
        "situacao_geral": situacao_geral
    }

    print("Perfeito, agora sim, pode me dizer qual a sua pergunta ou necessidade?")
    pergunta_inicial = input()

    return pergunta_inicial, informacoes_iniciais

In [71]:
'''
def direcionar_pergunta(pergunta, informacoes_iniciais, historico_conversa):
    """
    Analisa a pergunta do usuário e direciona para o agente especialista apropriado.
    """
    pergunta_lower = pergunta.lower()

    if any(keyword in pergunta_lower for keyword in ["advogado", "jurídico", "lei", "pensão", "guarda", "visita", "judicial", "processo"]):
        return agente_especialista_juridico(pergunta, informacoes_iniciais, historico_conversa)
    elif any(keyword in pergunta_lower for keyword in ["emocional", "sentimento", "psicológico", "terapia", "ansiedade", "depressão", "ajuda"]):
        return agente_especialista_psicologico(pergunta, informacoes_iniciais, historico_conversa)
    elif any(keyword in pergunta_lower for keyword in ["social", "recurso", "apoio", "creas", "cras", "assistência"]):
        return agente_especialista_social(pergunta, informacoes_iniciais, historico_conversa)
    elif any(keyword in pergunta_lower for keyword in ["negociação", "acordo", "comunicar", "diálogo", "conciliação"]):
        return agente_especialista_conciliador(pergunta, informacoes_iniciais, historico_conversa)
    else:
        # Se não identificar claramente, pode retornar para o agente geral inicialmente
        return agente_resposta_geral("Geral", pergunta, informacoes_iniciais, historico_conversa, "") # Passa um tópico genérico
'''


'\ndef direcionar_pergunta(pergunta, informacoes_iniciais, historico_conversa):\n    """\n    Analisa a pergunta do usuário e direciona para o agente especialista apropriado.\n    """\n    pergunta_lower = pergunta.lower()\n\n    if any(keyword in pergunta_lower for keyword in ["advogado", "jurídico", "lei", "pensão", "guarda", "visita", "judicial", "processo"]):\n        return agente_especialista_juridico(pergunta, informacoes_iniciais, historico_conversa)\n    elif any(keyword in pergunta_lower for keyword in ["emocional", "sentimento", "psicológico", "terapia", "ansiedade", "depressão", "ajuda"]):\n        return agente_especialista_psicologico(pergunta, informacoes_iniciais, historico_conversa)\n    elif any(keyword in pergunta_lower for keyword in ["social", "recurso", "apoio", "creas", "cras", "assistência"]):\n        return agente_especialista_social(pergunta, informacoes_iniciais, historico_conversa)\n    elif any(keyword in pergunta_lower for keyword in ["negociação", "acord

In [84]:
'''
from google.adk.tools import google_search
from datetime import date

def direcionar_pergunta(pergunta, informacoes_iniciais, historico_conversa):
    """
    Analisa a pergunta do usuário, aciona o Agente Pesquisador (google_search do adk) e direciona
    para o agente especialista apropriado com os resultados da pesquisa.
    """
    pergunta_lower = pergunta.lower()
    search_results = None
    topico = "Geral" # Tópico padrão
    hoje = date.today().strftime("%Y-%m-%d")

    if any(keyword in pergunta_lower for keyword in ["advogado", "jurídico", "lei", "pensão", "guarda", "visita", "judicial", "processo", "abandono"]):
        topico = "Jurídico"
        search_query = f"{pergunta} direito de família Brasil"
    elif any(keyword in pergunta_lower for keyword in ["emocional", "sentimento", "psicológico", "terapia", "ansiedade", "depressão", "ajuda"]):
        topico = "Psicológico"
        search_query = f"{pergunta} apoio psicológico para mães"
    elif any(keyword in pergunta_lower for keyword in ["social", "recurso", "apoio", "creas", "cras", "assistência"]):
        topico = "Social"
        search_query = f"{pergunta} assistência social para mães"
    elif any(keyword in pergunta_lower for keyword in ["negociação", "acordo", "comunicar", "diálogo", "conciliação"]):
        topico = "Conciliação e Negociação"
        search_query = f"{pergunta} mediação familiar"
    else:
        return agente_resposta_geral("Geral", pergunta, informacoes_iniciais, historico_conversa, "")

    if search_query:
        print(f"Pesquisando informações para '{topico}': {search_query}")
        # Chamando google_search diretamente como uma função
        search_results = google_search(query=search_query)
        print(f"Resultados da pesquisa para '{topico}': {search_results[:200]}...") # Exibe os primeiros 200 caracteres

    if topico == "Jurídico":
        return agente_especialista_juridico(pergunta, informacoes_iniciais, historico_conversa, search_results)
    elif topico == "Psicológico":
        return agente_especialista_psicologico(pergunta, informacoes_iniciais, historico_conversa, search_results)
    elif topico == "Social":
        return agente_especialista_social(pergunta, informacoes_iniciais, historico_conversa, search_results)
    elif topico == "Conciliação e Negociação":
        return agente_especialista_conciliador(pergunta, informacoes_iniciais, historico_conversa, search_results)

    return agente_resposta_geral("Geral", pergunta, informacoes_iniciais, historico_conversa, "")

'''

'\nfrom google.adk.tools import google_search\nfrom datetime import date\n\ndef direcionar_pergunta(pergunta, informacoes_iniciais, historico_conversa):\n    """\n    Analisa a pergunta do usuário, aciona o Agente Pesquisador (google_search do adk) e direciona\n    para o agente especialista apropriado com os resultados da pesquisa.\n    """\n    pergunta_lower = pergunta.lower()\n    search_results = None\n    topico = "Geral" # Tópico padrão\n    hoje = date.today().strftime("%Y-%m-%d")\n\n    if any(keyword in pergunta_lower for keyword in ["advogado", "jurídico", "lei", "pensão", "guarda", "visita", "judicial", "processo", "abandono"]):\n        topico = "Jurídico"\n        search_query = f"{pergunta} direito de família Brasil"\n    elif any(keyword in pergunta_lower for keyword in ["emocional", "sentimento", "psicológico", "terapia", "ansiedade", "depressão", "ajuda"]):\n        topico = "Psicológico"\n        search_query = f"{pergunta} apoio psicológico para mães"\n    elif any(

In [88]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date

def direcionar_pergunta(pergunta, informacoes_iniciais, historico_conversa):
    """
    Analisa a pergunta do usuário, aciona o Agente Pesquisador (google_search do adk)
    usando um agente e runner temporários, e direciona para o especialista.
    """
    pergunta_lower = pergunta.lower()
    search_results = None
    topico = "Geral"
    hoje = date.today().strftime("%Y-%m-%d")
    APP_NAME = "temporary_search_app"
    USER_ID = "temp_user"
    SESSION_ID = "temp_session"

    if any(keyword in pergunta_lower for keyword in ["advogado", "jurídico", "lei", "pensão", "guarda", "visita", "judicial", "processo", "abandono"]):
        topico = "Jurídico"
        search_query = f"{pergunta} direito de família Brasil"
    elif any(keyword in pergunta_lower for keyword in ["emocional", "sentimento", "psicológico", "terapia", "ansiedade", "depressão", "ajuda"]):
        topico = "Psicológico"
        search_query = f"{pergunta} apoio psicológico para mães"
    elif any(keyword in pergunta_lower for keyword in ["social", "recurso", "apoio", "creas", "cras", "assistência"]):
        topico = "Social"
        search_query = f"{pergunta} assistência social para mães"
    elif any(keyword in pergunta_lower for keyword in ["negociação", "acordo", "comunicar", "diálogo", "conciliação"]):
        topico = "Conciliação e Negociação"
        search_query = f"{pergunta} mediação familiar"
    else:
        return agente_resposta_geral("Geral", pergunta, informacoes_iniciais, historico_conversa, "")

    if search_query:
        print(f"Pesquisando informações para '{topico}': {search_query}")
        # Cria um agente temporário com a ferramenta de busca
        search_agent = Agent(
            name="temporary_search_agent",
            model="gemini-2.0-flash",
            tools=[google_search]
        )
        # Cria um runner temporário
        session_service = InMemorySessionService()
        session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
        runner = Runner(agent=search_agent, app_name=APP_NAME, session_service=session_service)

        # Executa a ferramenta de busca através do runner
        content = types.Content(role="user", parts=[types.Part(text=search_query)])
        events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
        for event in events:
            if event.is_final_response():
                search_results = event.content.parts[0].text
                print(f"Resultados da pesquisa para '{topico}': {search_results[:200]}...")
                break # Pega a primeira resposta final

    if topico == "Jurídico":
        return agente_especialista_juridico(pergunta, informacoes_iniciais, historico_conversa, search_results)
    elif topico == "Psicológico":
        return agente_especialista_psicologico(pergunta, informacoes_iniciais, historico_conversa, search_results)
    elif topico == "Social":
        return agente_especialista_social(pergunta, informacoes_iniciais, historico_conversa, search_results)
    elif topico == "Conciliação e Negociação":
        return agente_especialista_conciliador(pergunta, informacoes_iniciais, historico_conversa, search_results)

    return agente_resposta_geral("Geral", pergunta, informacoes_iniciais, historico_conversa, "")

In [ ]:
def main():
    """
    Função principal que inicia a conversa e mantém o loop de interação com o usuário.
    """
    historico_conversa = []
    pergunta_inicial, informacoes_iniciais = iniciar_conversa()

    pergunta_atual = pergunta_inicial

    while True:
        resposta_especialista = direcionar_pergunta(pergunta_atual, informacoes_iniciais, historico_conversa)
        resposta_cora_crua = agente_resposta_geral("Geral", pergunta_atual, informacoes_iniciais, historico_conversa, resposta_especialista) # Passa um tópico genérico
        resposta_final_revisada = agente_revisao_personalizacao(resposta_cora_crua, historico_conversa, informacoes_iniciais)

        print("\nCora diz:")
        print(resposta_final_revisada)

        historico_conversa.append({"usuario": pergunta_atual, "cora": resposta_final_revisada})

        continuar = input("\nVocê gostaria de continuar a conversa? (sim/não): ").lower()
        if continuar == "não":
            print("\nObrigada por conversar comigo! Se precisar de mais ajuda, estarei aqui.")
            break
        elif continuar != "sim":
            print("Por favor, responda apenas com 'sim' ou 'não'.")
            continue

        pergunta_atual = input("Qual a sua próxima pergunta? ")

In [89]:
if __name__ == "__main__":
    main()

Olá! Eu sou a Cora, sua assistente virtual dedicada a apoiar mães.
**Importante: As informações fornecidas por esta IA têm caráter informativo e de apoio, e não substituem a consulta com profissionais especializados (advogados, psicólogos, assistentes sociais, etc.). Em caso de necessidade, procure orientação profissional.**
Para te ajudar da melhor forma, gostaria de entender um pouco da sua situação.
Em que cidade você reside? ananideua
Qual a idade da criança envolvida? 7
Antes de me trazer sua dúvida gostaria de saber um pouco mais do contexto familiar, 
como é hoje a divisão de responsábilidades em relação a criação da criança? 100% eu
Perfeito, agora sim, pode me dizer qual a sua pergunta ou necessidade?
No ano passado eu e o pai dividiamos a criacao em 50% os custos financeiros ficavam em plano de saúde, remédios, roupas e dividiamos também em 50%. Ele decidiu mudar de estado e deixar a criaçao 100% na minha responsabilidade. Eu posso precessa-lo por abandono?
Pesquisando inform

In [90]:
'''
Essa versão está funcional.

'''

'\nEssa versão está funcional.\n\n'